In [1]:
import re
import string
import pandas as pd
import nltk
from nltk.collocations import *
from pattern.en import lemma
import requests
import json
import time
import hashlib
import uuid
import csv
import phrasemachine

# 1.Clean the text, lemma words

In [16]:
with open('The Heart Is a Lonely Hunter - Carson McCullers.txt', 'r',encoding='utf-8') as file:
    text = file.read().rstrip().lower()

In [17]:
# remove puntuation
word_list = re.findall("[a-z]+\-?\'?[a-z]+", text)

In [18]:
#Lemmatization
lemma_full_word_list = []
for i in word_list:
    word = lemma(i)
    lemma_full_word_list.append(word)

In [19]:
CET4_list = []

with open('CET4_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        if line.startswith(('a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z')):
            word = re.findall('([a-z]+)',line)
            CET4_list.append(word[0])
        else:
            continue

In [20]:
highschool_list = []

with open('Highschool_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        highschool_list.append(line.lower().rstrip())

In [21]:
elementary_list = []

with open('Elementary_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        highschool_list.append(line.lower().rstrip())

In [31]:
easy_word_list = set(CET4_list) | set(highschool_list) | set(elementary_list)
easy_word_list = list(easy_word_list)


In [23]:
#Lemmatize easy word list
lemma_easy_word_list = []
for i in easy_word_list:
    word = lemma(i)
    lemma_easy_word_list.append(word)

In [24]:
#select unfamiliar words
raw_word_list = []
for word in lemma_full_word_list:
    if word not in lemma_easy_word_list:
        raw_word_list.append(word)
    else:
        continue    

In [25]:
#clean '
raw_word_clean_list = []
for word in raw_word_list:
    word_clean = re.findall("[a-z]+\-?[a-z]+", word)[0]
    raw_word_clean_list.append(word_clean)

    

In [26]:
#count the word and write a csv
word_count = pd.value_counts(raw_word_clean_list)
data = pd.DataFrame({'Vocabulary':word_count.index, 'counts':word_count.values})
data.to_csv("vocabularycounts.csv",index=False,header=True)

In [27]:
data

,Vocabulary,counts
0,jake,230
1,portia,214
2,mick,214
3,copeland,205
4,biff,182
...,...,...
2953,fuz,1
2954,helper,1
2955,pouch,1
2956,tingle,1


# 2. Extact en translation and phonetics

#### 2.1 test

In [138]:
app_id = '7306b1b6'
app_key = '11013df45a50b3d7960bdcb4fe77af01'

language = 'en'
word_id = 'color'
fields = 'pronunciations'
strictMatch = 'false'

url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/' + language + '/' + word_id.lower() + '?fields=' + fields + '&strictMatch=' + strictMatch;

r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})

print("code {}\n".format(r.status_code))
print("text \n" + r.text)
print("json \n" + json.dumps(r.json()))

code 404

text 
{
    "error": "No entry found matching supplied source_lang, word and provided filters"
}
json 
{"error": "No entry found matching supplied source_lang, word and provided filters"}


In [79]:
a = r.json()
b = a["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']


'eɪs'

In [3]:
app_id = '7306b1b6'
app_key = '11013df45a50b3d7960bdcb4fe77af01'

language = 'en'
word_id = 'better'
fields = 'definitions'
strictMatch = 'false'

url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/' + language + '/' + word_id.lower() + '?fields=' + fields + '&strictMatch=' + strictMatch;

r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})


In [4]:
c = r.json()
d = c["results"][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions']
d

['more desirable, satisfactory, or effective']

In [110]:
# Youdao translation for plan B
youdao_url = 'https://openapi.youdao.com/api'  

input_text = "better"

time_curtime = int(time.time())   # 秒级时间戳获取
app_id = "15a1da979e8d68c0"   # 应用id
uu_id = uuid.uuid4()   # 随机生成的uuid数，为了每次都生成一个不重复的数。
app_key = "SSfsA7MhSvqvG79kKVzGx875O2ukapUW"   # 应用密钥

sign = hashlib.sha256((app_id + input_text + str(uu_id) + str(time_curtime) + app_key).encode('utf-8')).hexdigest()   # sign生成


data = {
    'q':translate_text,   # 翻译文本
    'from':"en",   # 源语言
    'to':"zh-CHS",   # 翻译语言
    'appKey':app_id,   # 应用id
    'salt':uu_id,   # 随机生产的uuid码
    'sign':sign,   # 签名
    'signType':"v3",   # 签名类型，固定值
    'curtime':time_curtime,   # 秒级时间戳
}



r = requests.get(youdao_url, params = data).json()
r['basic']['phonetic']

'ˈbetə(r)'

#### 2.2 the full version

In [26]:
csvFile = open("The Heart is a Lonely Hunter_VocabCount_v4.csv", "r")
reader = csv.reader(csvFile)

In [27]:
# API id and key
app_id = '7306b1b6'
app_key = '11013df45a50b3d7960bdcb4fe77af01'


In [28]:
word_pron = []   
word_pron_defin = []    

In [29]:
for i in reader:
    try:
        word_id = i[0]
        language = 'en'
        strictMatch = 'false'
        fields = 'pronunciations'

        url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/' + language + '/' + word_id.lower() + '?fields=' + fields + '&strictMatch=' + strictMatch;

        r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
        pron = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']
        word_pron.append([i[0].capitalize(),'/'+ pron + '/']) 
    except KeyError:
        pass
    continue

In [30]:
for i in word_pron:
    try:
        word_id = i[0]
        language = 'en'
        strictMatch = 'false'
        fields = 'definitions'
        url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/' + language + '/' + word_id.lower() + '?fields=' + fields + '&strictMatch=' + strictMatch;

        r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
        defin = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions'][0]
        i.append(defin)
        word_pron_defin.append(i)
    except KeyError:
        pass
    continue

In [48]:
fileObject = open('The Heart is a Lonely Hunter_VocabCount_v5.txt', 'w',encoding='utf-8')
for i in word_pron_defin:
    for j in i:
        fileObject.write(j)
        fileObject.write('|')
    fileObject.write('\n')
fileObject.close()

# 3. Find phrases in text

#### 3.1 test

In [104]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(word_list)
scored = finder.score_ngrams(bigram_measures.raw_freq)
sorted(bigram for bigram, score in scored)

[('-', '-lamming'),
 ('-', '-legs'),
 ('------', 'she'),
 ('-lamming', 'his'),
 ('-legs', 'and'),
 ('-t', 'or'),
 ('a', 'and'),
 ('a', 'animal'),
 ('a', 'b'),
 ('a', 'babble'),
 ('a', 'baby'),
 ('a', 'backfire'),
 ('a', 'backwash'),
 ('a', 'bad'),
 ('a', 'baggy'),
 ('a', 'ball'),
 ('a', 'band'),
 ('a', 'bandage'),
 ('a', 'banjo'),
 ('a', 'bar'),
 ('a', 'barbecue'),
 ('a', 'bare'),
 ('a', 'barefooted'),
 ('a', 'barking'),
 ('a', 'basket'),
 ('a', 'batch'),
 ('a', 'bath'),
 ('a', 'bathrobe'),
 ('a', 'battle'),
 ('a', 'bay'),
 ('a', 'beaming'),
 ('a', 'beard'),
 ('a', 'beautiful'),
 ('a', 'beauty'),
 ('a', 'bed'),
 ('a', 'bedbug'),
 ('a', 'bedlam'),
 ('a', 'bedroom'),
 ('a', 'bell'),
 ('a', 'bellboy'),
 ('a', 'bellyful'),
 ('a', 'bench'),
 ('a', 'berry'),
 ('a', 'better'),
 ('a', 'bible'),
 ('a', 'bicycle'),
 ('a', 'big'),
 ('a', 'bilious'),
 ('a', 'bill'),
 ('a', 'bird'),
 ('a', 'bit'),
 ('a', 'bitter'),
 ('a', 'black'),
 ('a', 'blacksmith'),
 ('a', 'bland'),
 ('a', 'blankness'),
 ('a', 

In [159]:
text = "The Omnibus Crime Control and Safe Streets Act of 1968 was signed into law by President Lyndon B. Johnson.so as James"
phrasemachine.get_phrases(text)

Exception: We don't support tagger nltk